In [553]:
import findspark
findspark.init()
findspark.find()
import pyspark

from pyspark.sql import SparkSession
import joblib
spark=SparkSession.builder.master("local").appName("TP").getOrCreate()

from pyspark.sql.functions import *

In [554]:
Dataset1 = spark.read.format("csv").option("sep",",").option("inferSchema", "true").option("header", "true").load("ccdefault.csv").withColumnRenamed("PAY_0", "PAY_1").withColumnRenamed("DEFAULT", "label")

In [555]:
from pyspark.sql.functions import isnan, when, count, col

In [556]:
Dataset1.select([count(when(isnan(c), c)).alias(c) for c in rawCreditCardDataset.columns]).show()

+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
| ID|LIMIT_BAL|SEX|EDUCATION|MARRIAGE|AGE|PAY_1|PAY_2|PAY_3|PAY_4|PAY_5|PAY_6|BILL_AMT1|BILL_AMT2|BILL_AMT3|BILL_AMT4|BILL_AMT5|BILL_AMT6|PAY_AMT1|PAY_AMT2|PAY_AMT3|PAY_AMT4|PAY_AMT5|PAY_AMT6|label|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+
|  0|        0|  0|        0|       0|  0|    0|    0|    0|    0|    0|    0|        0|        0|        0|        0|        0|        0|       0|       0|       0|       0|       0|       0|    0|
+---+---------+---+---------+--------+---+-----+-----+-----+-----+-----+-----+---------+---------+---------+---------+---------+---------+--------+--------+--------+--------+--------+--------+-----+



In [557]:
Dataset2 = Dataset1.withColumn("EDUCATION", when(col("EDUCATION") > 3, 0).otherwise(col("EDUCATION"))).withColumn("MARRIAGE", when(col("MARRIAGE") == 0, 3).otherwise(col("MARRIAGE")))

In [558]:
for i in range(1,7):
    colName = "PAY_"+str(i)
    Dataset2 = Dataset2.withColumn(colName, when(col(colName) < 0, 0).otherwise(col(colName)))

In [559]:
Dataset2.describe().show()

+-------+-----------------+------------------+------------------+------------------+------------------+-----------------+------------------+-------------------+-------------------+-------------------+------------------+-------------------+-----------------+-----------------+-----------------+------------------+-----------------+----------------+-----------------+------------------+-----------------+------------------+------------------+-----------------+-------------------+
|summary|               ID|         LIMIT_BAL|               SEX|         EDUCATION|          MARRIAGE|              AGE|             PAY_1|              PAY_2|              PAY_3|              PAY_4|             PAY_5|              PAY_6|        BILL_AMT1|        BILL_AMT2|        BILL_AMT3|         BILL_AMT4|        BILL_AMT5|       BILL_AMT6|         PAY_AMT1|          PAY_AMT2|         PAY_AMT3|          PAY_AMT4|          PAY_AMT5|         PAY_AMT6|              label|
+-------+-----------------+---------------

In [560]:
Dataset2.groupBy("EDUCATION").agg(avg("LIMIT_BAL").alias("LIMIT_BAL_MEAN")).show()

+---------+------------------+
|EDUCATION|    LIMIT_BAL_MEAN|
+---------+------------------+
|        1|212956.06991025034|
|        3|126550.27049013626|
|        2| 147062.4376336422|
|        0| 181316.2393162393|
+---------+------------------+



In [561]:
Dataset2.groupBy("label", "SEX").count().sort(asc("label"), asc("SEX")).show()

+-----+---+-----+
|label|SEX|count|
+-----+---+-----+
|    0|  1| 9015|
|    0|  2|14349|
|    1|  1| 2873|
|    1|  2| 3763|
+-----+---+-----+



In [562]:
Dataset2.describe("LIMIT_BAL", "AGE", "PAY_1").show()

+-------+------------------+-----------------+------------------+
|summary|         LIMIT_BAL|              AGE|             PAY_1|
+-------+------------------+-----------------+------------------+
|  count|             30000|            30000|             30000|
|   mean|167484.32266666667|          35.4855|0.3567666666666667|
| stddev|129747.66156720246|9.217904068090155|0.7605941727834076|
|    min|             10000|               21|                 0|
|    max|           1000000|               79|                 8|
+-------+------------------+-----------------+------------------+



In [563]:
Dataset2.groupBy("label", "MARRIAGE").count().sort(asc("label"), asc("MARRIAGE")).show()

+-----+--------+-----+
|label|MARRIAGE|count|
+-----+--------+-----+
|    0|       1|10453|
|    0|       2|12623|
|    0|       3|  288|
|    1|       1| 3206|
|    1|       2| 3341|
|    1|       3|   89|
+-----+--------+-----+



In [564]:
Dataset2.groupBy("label", "EDUCATION").count().sort(asc("label"), asc("EDUCATION")).show()

+-----+---------+-----+
|label|EDUCATION|count|
+-----+---------+-----+
|    0|        0|  435|
|    0|        1| 8549|
|    0|        2|10700|
|    0|        3| 3680|
|    1|        0|   33|
|    1|        1| 2036|
|    1|        2| 3330|
|    1|        3| 1237|
+-----+---------+-----+



In [565]:
from pyspark.ml.feature import VectorAssembler,StandardScaler
from pyspark.ml.feature import OneHotEncoder
va = VectorAssembler().setInputCols(["LIMIT_BAL", "AGE", "EDUCATION", "SEX", "MARRIAGE", "label"]).setOutputCol("feature")                                 

In [566]:
numericColonn = ["LIMIT_BAL"]

In [567]:
categoricColonns = ["EDUCATION", "SEX", "MARRIAGE", "PAY_1", "PAY_2", "PAY_3", "PAY_4", "PAY_5", "PAY_6"]

In [568]:
categoricColumnsOhe = list(map(lambda x :x+"_OHE",categoricColumns))

In [569]:
va = VectorAssembler().setInputCols(numericColonn).setOutputCol("numerical_attributes")

In [570]:
scaler = StandardScaler().setInputCol("numerical_attributes").setOutputCol("scaled_attributes")

In [571]:
encoder = OneHotEncoder().setInputCols(categoricColonns).setOutputCols(categoricColonnsOhe)

In [572]:
from pyspark.ml import Pipeline, PipelineModel

In [573]:
numPipeline =  Pipeline().setStages([va, scaler])
catPipeline =  Pipeline().setStages([encoder])
pipeline =  Pipeline().setStages([numPipeline, catPipeline])

In [574]:
DatasetF = pipeline.fit(creditCardDataset).transform(creditCardDataset)

In [575]:
datasetFINAL = VectorAssembler().setInputCols([("scaled_attributes")] + categoricColonnsOhe).setOutputCol("features").transform(DatasetF).select("features", "label")

In [576]:
datasetFINAL.show(5, False)

+-----------------------------------------------------------------------------------------+-----+
|features                                                                                 |label|
+-----------------------------------------------------------------------------------------+-----+
|(57,[0,3,7,11,19,25,33,41,49],[0.15414535998894324,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])     |1    |
|(57,[0,3,8,9,19,25,33,41,51],[0.9248721599336596,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |1    |
|(57,[0,3,8,9,17,25,33,41,49],[0.6936541199502446,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])       |0    |
|(57,[0,3,7,9,17,25,33,41,49],[0.38536339997235813,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])      |0    |
|(57,[0,3,5,7,9,17,25,33,41,49],[0.38536339997235813,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0])|0    |
+-----------------------------------------------------------------------------------------+-----+
only showing top 5 rows



In [577]:
trainSet, testSet = datasetFINAL.randomSplit([0.8, 0.2])

In [578]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.evaluation import BinaryClassificationMetrics
from pyspark.mllib.evaluation import MulticlassMetrics
from pyspark.ml.tuning import ParamGridBuilder
from pyspark.ml.tuning import CrossValidator
from pyspark.sql import DataFrame
from pyspark.mllib.regression import LabeledPoint

In [579]:
lr = LogisticRegression().setMaxIter(10)

In [580]:
paramGrid = ParamGridBuilder().addGrid(lr.regParam,[0.2, 0.1, 0.0]).addGrid(lr.elasticNetParam,[0.2, 0.1, 0.0]).build()

In [581]:
evaluator = BinaryClassificationEvaluator()

In [582]:
v = CrossValidator().setEstimator(lr).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(4)

In [583]:
lrModel = v.fit(trainSet)

In [584]:
##print(lrModel.bestModel.extractParamMap())

In [585]:
predictions = lrModel.transform(testSet)

In [586]:
def compute_metrics(predictions: pyspark.sql.DataFrame):
    predictionAndLabels = predictions.select("prediction", "label").rdd
    metrics = BinaryClassificationMetrics(predictionAndLabels)
    print("Area under ROC:" %{metrics.areaUnderPR})
    print("Accuracy =" %{metrics.accuracy} )
    print("F-measure =" %{metrics.fMeasure(1)} )
    print("Confusion matrix: ")
    print(mMetrics.confusionMatrix)

In [587]:
predictions: pyspark.sql.DataFrame
predictionAndLabels = predictions.select("prediction", "label").rdd
metrics = BinaryClassificationMetrics(predictionAndLabels)
##print("Area under PR =" %metrics.areaUnderPR)

In [588]:
metrics =BinaryClassificationMetrics(predictions.select("prediction", "label").rdd)

In [589]:
print("Area under PR = %s" %metrics.areaUnderPR)

Py4JJavaError: An error occurred while calling o183400.areaUnderPR.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 21054.0 failed 1 times, most recent failure: Lost task 0.0 in stage 21054.0 (TID 21839) (DESKTOP-IPHB0ET executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 604, in main
  File "C:\spark\python\lib\pyspark.zip\pyspark\worker.py", line 596, in process
  File "C:\spark\python\lib\pyspark.zip\pyspark\serializers.py", line 259, in dump_stream
    vs = list(itertools.islice(iterator, batch))
  File "C:\spark\python\lib\pyspark.zip\pyspark\util.py", line 73, in wrapper
    return f(*args, **kwargs)
  File "C:\spark\python\pyspark\sql\session.py", line 682, in prepare
    verify_func(obj)
  File "C:\spark\python\pyspark\sql\types.py", line 1409, in verify
    verify_value(obj)
  File "C:\spark\python\pyspark\sql\types.py", line 1390,

In [ ]:
print(lrModel.subModels)

In [ ]:
from pyspark.ml.classification import DecisionTreeClassifier

In [ ]:
dt = DecisionTreeClassifier()

In [ ]:
paramGrid =ParamGridBuilder().addGrid(dt.maxDepth, [4, 5, 6, 7, 8, 9, 10]).build()

In [ ]:
evaluator = BinaryClassificationEvaluator()

In [ ]:
cv = CrossValidator().setEstimator(dt).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(4)

In [ ]:
dtModel = cv.fit(trainSet)

In [ ]:
predictions = dtModel.transform(testSet)

In [ ]:
compute_metrics(predictions)

In [ ]:
from pyspark.ml.classification import RandomForestClassifier

In [331]:
rf = RandomForestClassifier()

In [339]:
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth,[10, 15]).addGrid(rf.featureSubsetStrategy,["all", "sqrt"]).addGrid(rf.numTrees,[10, 15,20]).build()

In [340]:
valuator = BinaryClassificationEvaluator()

In [342]:
cv = CrossValidator().setEstimator(rf).setEstimatorParamMaps(paramGrid).setEvaluator(evaluator).setNumFolds(4)

In [343]:
rfModel = cv.fit(trainSet)

In [344]:
predictions = rfModel.transform(testSet)

In [552]:
predictionAndLabels = testSet.map(lambda lp:(float(rfModel.predict(lp.prediction)), lp.label))

AttributeError: 'DataFrame' object has no attribute 'map'